## Classification of News Articles

In [1]:
#Importing Libraries
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

## Using Spacy's small language model

In [4]:
!pip install spacy

In [5]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [6]:
df = pd.read_csv('BBC News.csv')
df.head()

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ArticleId  1490 non-null   int64 
 1   Text       1490 non-null   object
 2   Category   1490 non-null   object
dtypes: int64(1), object(2)
memory usage: 35.0+ KB


In [8]:
df.drop('ArticleId', axis=1, inplace=True)

In [10]:
def is_whitespace(data):
    
    blank = []
    for idx, text, label in data.itertuples():
        if text.isspace():
            blank.append(idx)
    
    return blank

In [11]:
is_whitespace(df)

[]

In [12]:
df.Text[0]

'worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.  cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy.  prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002. she said andersen had given a  green light  to the procedures and practices used by worldcom. mr ebb

## Preprocessing using spacy

In [13]:
def preprocess(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct or token.is_space:
            continue
        filtered_tokens.append(token.lemma_)
    
    return " ".join(filtered_tokens)

Removing stop words, punctuations from the text.

In [14]:
df['processed_text'] = df['Text'].apply(preprocess)

In [15]:
df.head()

,Text,Category,processed_text
0,worldcom ex-boss launches defence lawyers defe...,business,worldcom ex boss launch defence lawyer defend ...
1,german business confidence slides german busin...,business,german business confidence slide german busine...
2,bbc poll indicates economic gloom citizens in ...,business,bbc poll indicate economic gloom citizen major...
3,lifestyle governs mobile choice faster bett...,tech,lifestyle govern mobile choice fast well funky...
4,enron bosses in $168m payout eighteen former e...,business,enron boss $ 168 m payout eighteen enron direc...


In [16]:
df.Text[0]

'worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.  cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy.  prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002. she said andersen had given a  green light  to the procedures and practices used by worldcom. mr ebb

In [17]:
df.processed_text[0]

'worldcom ex boss launch defence lawyer defend worldcom chief bernie ebber battery fraud charge call company whistleblow witness cynthia cooper worldcom s ex head internal accounting alert director irregular accounting practice telecoms giant 2002 warning lead collapse firm follow discovery $ 11bn £ 5.7bn accounting fraud mr ebber plead guilty charge fraud conspiracy prosecution lawyer argue mr ebber orchestrate series accounting trick worldcom order employee hide expense inflate revenue meet wall street earning estimate ms cooper run consulting business tell jury new york wednesday external auditor arthur andersen approve worldcom s accounting early 2001 2002 say andersen give green light procedure practice worldcom mr ebber s lawyer say unaware fraud argue auditor alert problem ms cooper say shareholder meeting mr ebber pass technical question company s finance chief give brief answer prosecution s star witness worldcom financial chief scott sullivan say mr ebber order accounting adj

## Encoding and Splitting

In [18]:
le = LabelEncoder()
cat_fit = le.fit(df.Category)
y = cat_fit.transform(df.Category)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(df.processed_text, y, 
                                                    test_size=0.2, random_state=42)

In [20]:
X_train.shape, X_test.shape

((1192,), (298,))

In [21]:
y_train.shape, y_test.shape

((1192,), (298,))

## Modelling and evaluation

Used sklearn pipelines to perform preprocessing and modelling in sequence.

## Model 1 using
  CountVectorizer: 1-gram and bi-gram
  
  
  Naive Bayes algorithm

In [22]:
model_1 = Pipeline([('c_vectorizer', CountVectorizer(ngram_range=(1, 2))), 
                      ('bayes_model', MultinomialNB())])

In [23]:
model_1.fit(X_train, y_train)

Pipeline(steps=[('c_vectorizer', CountVectorizer(ngram_range=(1, 2))),
                ('bayes_model', MultinomialNB())])

In [24]:
model_1_pred = model_1.predict(X_test)

In [25]:
print(f'\nAccuracy score of count vectorizer based model: {accuracy_score(y_test, model_1_pred):.2f}')



Accuracy score of count vectorizer based model: 0.98


In [26]:
model_2 = Pipeline([('t_vector', TfidfVectorizer()), 
                    ('bayes_model_2', MultinomialNB())])

In [27]:
model_2.fit(X_train, y_train)

Pipeline(steps=[('t_vector', TfidfVectorizer()),
                ('bayes_model_2', MultinomialNB())])

In [28]:
model_2_pred = model_2.predict(X_test)


In [29]:
print(f'\nAccuracy score of tfidf based model: {accuracy_score(y_test, model_2_pred):.2f}')


Accuracy score of tfidf based model: 0.96


## In the case of this data:

It appears countvectorizer with uni-gram and bi-gram range gives a better result: 98% accuracy.
Compared to this the tfidf vectorizer based method only provided